In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import sklearn
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from sklearn import feature_extraction,model_selection
from sklearn.metrics import confusion_matrix
import glob
import os
import string
import tensorflow as tf
import keras as K
import zipfile
from itertools import chain
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix, classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
# importing Word2Vec
from gensim.models import Word2Vec
from gensim.models import FastText

In [5]:
X=pd.read_csv('/content/hindi_tr.csv')
X.head()

,tweet_id,text,task1,task2,ID
0,1.127755e+18,1 आदमीं को मारने पर गोडसे आतंकी हो सके है तो\n...,HOF,HATE,hasoc_2020_hi_2250
1,1.123579e+18,"RT @Vishesh4: @jawaharyadavbjp जवाहर यादव, अगर...",NOT,NONE,hasoc_2020_hi_381
2,1.127750e+18,RT @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही ...,HOF,HATE,hasoc_2020_hi_1510
3,1.127661e+18,Yey nina khothani labafazi benu phambili Finis...,HOF,PRFN,hasoc_2020_hi_1010
4,1.123487e+18,RT @Rajeshbhanjan2: जब भी कोई सिकुलर कोंग्रेसी...,HOF,HATE,hasoc_2020_hi_331


In [6]:
X=X[['text','task1']]
X

,text,task1
0,1 आदमीं को मारने पर गोडसे आतंकी हो सके है तो\n...,HOF
1,"RT @Vishesh4: @jawaharyadavbjp जवाहर यादव, अगर...",NOT
2,RT @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही ...,HOF
3,Yey nina khothani labafazi benu phambili Finis...,HOF
4,RT @Rajeshbhanjan2: जब भी कोई सिकुलर कोंग्रेसी...,HOF
...,...,...
2958,@Varun_dvn जिसके नाम के साथ इंडिअनस लगा हो उसक...,NOT
2959,मुझे सबसे ज्यादा राहुल और सोनियाके बारे बुरा ल...,NOT
2960,RT @Gujju_Er: एक आतंकी जिसे कांग्रेस ने\n'शांत...,HOF
2961,RT @dayanand151: @jafahar1 @BBCHindi @Hariramk...,HOF


In [7]:
df1=pd.read_csv('/content/hindi_test_1509.csv')
df1.head()

,tweet_id,text,task1,task2,ID
0,1127781647783301120,@Kinjal_Dubey_ इसके लिए तैयारी हो रखी है\nआपको...,HOF,OFFN,hasoc_2020_hi_1938
1,1127492853171535872,RT @pratimamishra04: दिल्ली के मुख्यमंत्री @Ar...,NOT,NONE,hasoc_2020_hi_1201
2,1123638375896903681,"RT @nishantdabre2: आज एक तरफ खुशी है, तो एक तर...",NOT,NONE,hasoc_2020_hi_1628
3,1127496460310683648,@BJP4India @HardeepSPuri जिस कुनबे के लोग :-\n...,NOT,NONE,hasoc_2020_hi_3257
4,1127667277535399937,@manakgupta बादल....😂😂😂,NOT,NONE,hasoc_2020_hi_4133


In [8]:
Y=df1[['text','task1']]
Y

,text,task1
0,@Kinjal_Dubey_ इसके लिए तैयारी हो रखी है\nआपको...,HOF
1,RT @pratimamishra04: दिल्ली के मुख्यमंत्री @Ar...,NOT
2,"RT @nishantdabre2: आज एक तरफ खुशी है, तो एक तर...",NOT
3,@BJP4India @HardeepSPuri जिस कुनबे के लोग :-\n...,NOT
4,@manakgupta बादल....😂😂😂,NOT
...,...,...
658,RT @AishwaryVerma9: दिल्ली में शून्य थे और शून...,NOT
659,RT @KARUNASHANKEROJ: @sagarikaghose एक आर्टिकल...,HOF
660,@CM_P17 @PShatrujeet रमजान शुरू लेकिन मजाल है ...,HOF
661,RT @rukasa007: तो तुम्हारे सिर में एक गोली उता...,NOT


In [9]:
X.head()

,text,task1
0,1 आदमीं को मारने पर गोडसे आतंकी हो सके है तो\n...,HOF
1,"RT @Vishesh4: @jawaharyadavbjp जवाहर यादव, अगर...",NOT
2,RT @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही ...,HOF
3,Yey nina khothani labafazi benu phambili Finis...,HOF
4,RT @Rajeshbhanjan2: जब भी कोई सिकुलर कोंग्रेसी...,HOF


In [10]:
cols=['task1']
X[cols] = X[cols].apply(lambda x: pd.factorize(x)[0] + 0)
Y[cols] = Y[cols].apply(lambda x: pd.factorize(x)[0] + 0)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [11]:
X.head()

,text,task1
0,1 आदमीं को मारने पर गोडसे आतंकी हो सके है तो\n...,0
1,"RT @Vishesh4: @jawaharyadavbjp जवाहर यादव, अगर...",1
2,RT @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही ...,0
3,Yey nina khothani labafazi benu phambili Finis...,0
4,RT @Rajeshbhanjan2: जब भी कोई सिकुलर कोंग्रेसी...,0


In [12]:
Y.head()

,text,task1
0,@Kinjal_Dubey_ इसके लिए तैयारी हो रखी है\nआपको...,0
1,RT @pratimamishra04: दिल्ली के मुख्यमंत्री @Ar...,1
2,"RT @nishantdabre2: आज एक तरफ खुशी है, तो एक तर...",1
3,@BJP4India @HardeepSPuri जिस कुनबे के लोग :-\n...,1
4,@manakgupta बादल....😂😂😂,1


In [13]:
X_train=X['text']
X_train

0       1 आदमीं को मारने पर गोडसे आतंकी हो सके है तो\n...
1       RT @Vishesh4: @jawaharyadavbjp जवाहर यादव, अगर...
2       RT @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही ...
3       Yey nina khothani labafazi benu phambili Finis...
4       RT @Rajeshbhanjan2: जब भी कोई सिकुलर कोंग्रेसी...
                              ...                        
2958    @Varun_dvn जिसके नाम के साथ इंडिअनस लगा हो उसक...
2959    मुझे सबसे ज्यादा राहुल और सोनियाके बारे बुरा ल...
2960    RT @Gujju_Er: एक आतंकी जिसे कांग्रेस ने\n'शांत...
2961    RT @dayanand151: @jafahar1 @BBCHindi @Hariramk...
2962    RT @manakgupta: “हमें 5 साल हो गए, हमने संविधा...
Name: text, Length: 2963, dtype: object

In [14]:
y_train=X['task1']
y_train

0       0
1       1
2       0
3       0
4       0
       ..
2958    1
2959    1
2960    0
2961    0
2962    1
Name: task1, Length: 2963, dtype: int64

In [15]:
X_test=Y['text']
X_test

0      @Kinjal_Dubey_ इसके लिए तैयारी हो रखी है\nआपको...
1      RT @pratimamishra04: दिल्ली के मुख्यमंत्री @Ar...
2      RT @nishantdabre2: आज एक तरफ खुशी है, तो एक तर...
3      @BJP4India @HardeepSPuri जिस कुनबे के लोग :-\n...
4                                @manakgupta बादल....😂😂😂
                             ...                        
658    RT @AishwaryVerma9: दिल्ली में शून्य थे और शून...
659    RT @KARUNASHANKEROJ: @sagarikaghose एक आर्टिकल...
660    @CM_P17 @PShatrujeet रमजान शुरू लेकिन मजाल है ...
661    RT @rukasa007: तो तुम्हारे सिर में एक गोली उता...
662    @athavale_abhi @kavita_tewari गांधी परिवार और ...
Name: text, Length: 663, dtype: object

In [16]:
y_test=Y['task1']
y_test

0      0
1      1
2      1
3      1
4      1
      ..
658    1
659    0
660    0
661    1
662    0
Name: task1, Length: 663, dtype: int64

In [17]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((2963,), (663,), (2963,), (663,))

In [18]:
def create_tokens(lines):
  #stop_words = set(nltk.corpus.stopwords.words('english'))
  t_line=lines.apply(lambda x: nltk.word_tokenize(x))
  #t_line=t_line.apply(lambda x: [w for w in x if not w in stop_words])
  return t_line

In [19]:
X_train=create_tokens(X_train)
print(X_train)
tokenizer= Tokenizer()
tokenizer.fit_on_texts(X_train)
tokenizer


0       [1, आदमीं, को, मारने, पर, गोडसे, आतंकी, हो, सक...
1       [RT, @, Vishesh4, :, @, jawaharyadavbjp, जवाहर...
2       [RT, @, FunKeyBaat, :, #, भगवा, वस्त्र, पहन, क...
3       [Yey, nina, khothani, labafazi, benu, phambili...
4       [RT, @, Rajeshbhanjan2, :, जब, भी, कोई, सिकुलर...
                              ...                        
2958    [@, Varun_dvn, जिसके, नाम, के, साथ, इंडिअनस, ल...
2959    [मुझे, सबसे, ज्यादा, राहुल, और, सोनियाके, बारे...
2960    [RT, @, Gujju_Er, :, एक, आतंकी, जिसे, कांग्रेस...
2961    [RT, @, dayanand151, :, @, jafahar1, @, BBCHin...
2962    [RT, @, manakgupta, :, “, हमें, 5, साल, हो, गए...
Name: text, Length: 2963, dtype: object


In [20]:
length=np.average(X_train.apply(lambda x:len(x)*len(x)))
length=np.sqrt(length)
print(length)
length=48

29.573449750382068


In [21]:
def encode_text(tokenizer, lines, length):
  encoded = tokenizer.texts_to_sequences(lines)
  padded = pad_sequences(encoded, maxlen=length, padding='post')
  return padded

In [22]:
word_index=tokenizer.word_index

In [23]:
X_test=create_tokens(X_test)
print(X_test)

0      [@, Kinjal_Dubey_, इसके, लिए, तैयारी, हो, रखी,...
1      [RT, @, pratimamishra04, :, दिल्ली, के, मुख्यम...
2      [RT, @, nishantdabre2, :, आज, एक, तरफ, खुशी, ह...
3      [@, BJP4India, @, HardeepSPuri, जिस, कुनबे, के...
4                       [@, manakgupta, बादल, ..., .😂😂😂]
                             ...                        
658    [RT, @, AishwaryVerma9, :, दिल्ली, में, शून्य,...
659    [RT, @, KARUNASHANKEROJ, :, @, sagarikaghose, ...
660    [@, CM_P17, @, PShatrujeet, रमजान, शुरू, लेकिन...
661    [RT, @, rukasa007, :, तो, तुम्हारे, सिर, में, ...
662    [@, athavale_abhi, @, kavita_tewari, गांधी, पर...
Name: text, Length: 663, dtype: object


In [24]:
X_train=encode_text(tokenizer,X_train,length)
X_train
#pd.DataFrame(train_x).to_csv("train_file.csv")

array([[  258,  4967,     9, ...,     0,     0,     0],
       [    4,     1,  4973, ...,     0,     0,     0],
       [    4,     1,  1497, ...,     0,     0,     0],
       ...,
       [    4,     1, 14733, ...,     0,     0,     0],
       [    4,     1, 14736, ...,     0,     0,     0],
       [    4,     1,   641, ...,     0,     0,     0]], dtype=int32)

In [25]:
X_test=encode_text(tokenizer,X_test,length)
X_test
#pd.DataFrame(train_x).to_csv("train_file.csv")

array([[   1,  299,   40, ...,    0,    0,    0],
       [   4,    1,    2, ...,    0,    0,    0],
       [   4,    1,    2, ...,    0,    0,    0],
       ...,
       [   1,    1, 4755, ...,    0,    0,    0],
       [   4,    1, 1219, ...,    0,    0,    0],
       [   1,    1,  144, ...,    0,    0,    0]], dtype=int32)

In [26]:
from keras.utils.np_utils import to_categorical
y_test = to_categorical(np.asarray(y_test))
y_train = to_categorical(np.asarray(y_train))
print(y_test)
print(y_train)

[[1. 0.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]
[[1. 0.]
 [0. 1.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [0. 1.]]


In [27]:
# print(len(X_train))
# print(len(X_test))
# print(len(y_train))
# print(len(y_test))
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((2963, 48), (663, 48), (2963, 2), (663, 2))

In [29]:
# Creating embedding matrix
# Using 
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/My Drive/Copy of crawl-300d-2M.vec.zip", 'r')
zip_ref.extractall()
zip_ref.close()

In [30]:
#load pre-trained embedding:
embeddings_index = {}
f = open('/content/crawl-300d-2M.vec', encoding="utf8")
for line in f:
    values = line.split()#This splits each values in a  line with spaces
    word = values[0] # This store the first values which is the word in english
    coefs = np.asarray(values[1:], dtype='float32') # This converts all splitted values from 1 to last(expect first word) into an array
    embeddings_index[word] = coefs # english word = array of numerical values
f.close()
print('Total %s word vectors in crawl-300-2M.' % len(embeddings_index))

Total 1999996 word vectors in crawl-300-2M.


In [31]:
print((X_train))
print((X_test))
print((y_train))
print((y_test))

[[  258  4967     9 ...     0     0     0]
 [    4     1  4973 ...     0     0     0]
 [    4     1  1497 ...     0     0     0]
 ...
 [    4     1 14733 ...     0     0     0]
 [    4     1 14736 ...     0     0     0]
 [    4     1   641 ...     0     0     0]]
[[   1  299   40 ...    0    0    0]
 [   4    1    2 ...    0    0    0]
 [   4    1    2 ...    0    0    0]
 ...
 [   1    1 4755 ...    0    0    0]
 [   4    1 1219 ...    0    0    0]
 [   1    1  144 ...    0    0    0]]
[[1. 0.]
 [0. 1.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [0. 1.]]
[[1. 0.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]


In [33]:
# Initialize the weight for each word for the embedding layer. Words not present
# in the dictionary initialized by random value 
EMBEDDING_DIM = 300
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
#embedding_matrix = (16433, 100) NOTE: It is from uniques words in our tweets
for word, i in word_index.items(): # Selecting word from our tweet unique words list for changing with the values from glove
    embedding_vector = embeddings_index.get(word) # Finding our tweet words in glove and taking its array of 100 values
    if embedding_vector is not None: 
#words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
embedding_matrix.shape

(14743, 300)

In [34]:
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Input,LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
model=Sequential()
model.add(Embedding(input_dim = 14743, output_dim = 300, weights=[embedding_matrix], input_length=length,trainable=False))
model.add(LSTM(192, dropout=0.3, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 48, 300)           4422900   
_________________________________________________________________
lstm (LSTM)                  (None, 192)               378624    
_________________________________________________________________
dense (Dense)                (None, 2)                 386       
Total params: 4,801,910
Trainable params: 379,010
Non-trainable params: 4,422,900
_________________________________________________________________


In [35]:
model.fit(X_train, y_train, validation_data = (X_test, y_test),epochs=100,batch_size=256, verbose=2)

Epoch 1/100
12/12 - 2s - loss: 0.7450 - accuracy: 0.6797 - val_loss: 0.6295 - val_accuracy: 0.7074
Epoch 2/100
12/12 - 2s - loss: 0.6090 - accuracy: 0.7135 - val_loss: 0.6081 - val_accuracy: 0.7029
Epoch 3/100
12/12 - 2s - loss: 0.5977 - accuracy: 0.7148 - val_loss: 0.6044 - val_accuracy: 0.7029
Epoch 4/100
12/12 - 2s - loss: 0.5946 - accuracy: 0.7145 - val_loss: 0.6062 - val_accuracy: 0.7029
Epoch 5/100
12/12 - 2s - loss: 0.5949 - accuracy: 0.7145 - val_loss: 0.6035 - val_accuracy: 0.7029
Epoch 6/100
12/12 - 2s - loss: 0.5962 - accuracy: 0.7148 - val_loss: 0.6050 - val_accuracy: 0.7029
Epoch 7/100
12/12 - 2s - loss: 0.5960 - accuracy: 0.7155 - val_loss: 0.6075 - val_accuracy: 0.7029
Epoch 8/100
12/12 - 2s - loss: 0.5949 - accuracy: 0.7168 - val_loss: 0.6034 - val_accuracy: 0.7014
Epoch 9/100
12/12 - 2s - loss: 0.5939 - accuracy: 0.7179 - val_loss: 0.6083 - val_accuracy: 0.7029
Epoch 10/100
12/12 - 2s - loss: 0.5932 - accuracy: 0.7165 - val_loss: 0.6057 - val_accuracy: 0.7029
Epoch 11/

In [36]:
predictions_test = model.predict(X_test)

predictions_test

array([[2.4004277e-02, 9.7599572e-01],
       [2.1958204e-03, 9.9780422e-01],
       [5.5064507e-02, 9.4493550e-01],
       ...,
       [3.3831887e-04, 9.9966168e-01],
       [8.7119287e-01, 1.2880717e-01],
       [4.8772740e-04, 9.9951231e-01]], dtype=float32)

In [37]:
pred_test=np.zeros_like(predictions_test)
pred_test[np.arange(len(predictions_test)),predictions_test.argmax(1)]=1
# output_DataFrame=pd.DataFrame(pred_test.argmax(axis=1),index=df1.index)
# output_DataFrame[0]=output_DataFrame[0].map({0:'NAG',1:'CAG',2:'OAG'})
# output_DataFrame.to_csv("Hindi Subtask B ou.csv")

In [38]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred_test))

              precision    recall  f1-score   support

           0       0.43      0.24      0.31       197
           1       0.73      0.86      0.79       466

   micro avg       0.68      0.68      0.68       663
   macro avg       0.58      0.55      0.55       663
weighted avg       0.64      0.68      0.65       663
 samples avg       0.68      0.68      0.68       663



In [39]:
  df3=df1[['tweet_id','ID']]
  df3

,tweet_id,ID
0,1127781647783301120,hasoc_2020_hi_1938
1,1127492853171535872,hasoc_2020_hi_1201
2,1123638375896903681,hasoc_2020_hi_1628
3,1127496460310683648,hasoc_2020_hi_3257
4,1127667277535399937,hasoc_2020_hi_4133
...,...,...
658,1123814066907242503,hasoc_2020_hi_466
659,1127616035689979904,hasoc_2020_hi_1758
660,1127502823044702208,hasoc_2020_hi_3521
661,1127780829914992640,hasoc_2020_hi_1293


In [40]:
predictions_test1=predictions_test.argmax(1)
pred_test1=pd.Series(predictions_test1)
pred_test1

0      1
1      1
2      1
3      1
4      1
      ..
658    1
659    1
660    1
661    0
662    1
Length: 663, dtype: int64

In [41]:
df3['label']=pred_test1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [42]:
df3

,tweet_id,ID,label
0,1127781647783301120,hasoc_2020_hi_1938,1
1,1127492853171535872,hasoc_2020_hi_1201,1
2,1123638375896903681,hasoc_2020_hi_1628,1
3,1127496460310683648,hasoc_2020_hi_3257,1
4,1127667277535399937,hasoc_2020_hi_4133,1
...,...,...,...
658,1123814066907242503,hasoc_2020_hi_466,1
659,1127616035689979904,hasoc_2020_hi_1758,1
660,1127502823044702208,hasoc_2020_hi_3521,1
661,1127780829914992640,hasoc_2020_hi_1293,0


In [43]:
df3['label'] = df3['label'].map({0: "NOT",1:"HOF"})
df3

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,tweet_id,ID,label
0,1127781647783301120,hasoc_2020_hi_1938,HOF
1,1127492853171535872,hasoc_2020_hi_1201,HOF
2,1123638375896903681,hasoc_2020_hi_1628,HOF
3,1127496460310683648,hasoc_2020_hi_3257,HOF
4,1127667277535399937,hasoc_2020_hi_4133,HOF
...,...,...,...
658,1123814066907242503,hasoc_2020_hi_466,HOF
659,1127616035689979904,hasoc_2020_hi_1758,HOF
660,1127502823044702208,hasoc_2020_hi_3521,HOF
661,1127780829914992640,hasoc_2020_hi_1293,NOT


In [45]:
df3.to_csv('submission_HI_A.csv')

In [46]:
model.save("HindiDatasetSubTask1_withLSTM_Fasttext.h5")